In [1]:
from sqlalchemy import create_engine

import getpass
import sqlalchemy as sa

## Settings

Replace these parameters

In [31]:
CDM_SCHEMA = ''  # 'erspc'
RESULTS_SCHEMA = ''  # 'erspc_results'
CDM_DATA_PATH = ''  # '../data/erspc_omop_cdm.sql'

# Connect to the db

In [2]:
server = 'postgresql'
port = '5432'
db = 'ohdsi'
engine = create_engine(f'postgresql://{input("User:")}:{getpass.getpass("Password:")}@{server}:{port}/{db}')

User: admin
Password: ·············


In [171]:
#sa.inspect(engine).get_schema_names()

['erspc',
 'erspc_results',
 'erspc_synthetic',
 'erspc_synthetic_results',
 'information_schema',
 'ohdsi',
 'public',
 'synpuf_1k',
 'synpuf_1k_results',
 'temp',
 'vocab']

In [29]:
#sa.inspect(engine).get_table_names('erspc_results')

['cohort',
 'cohort_features',
 'cohort_features_analysis_ref',
 'cohort_features_dist',
 'cohort_features_ref',
 'achilles_results_dist',
 'cohort_inclusion',
 'cohort_inclusion_result',
 'cohort_inclusion_stats',
 'cohort_summary_stats',
 'cohort_censor_stats',
 'feas_study_inclusion_stats',
 'feas_study_index_stats',
 'feas_study_result',
 'heracles_analysis',
 'heracles_heel_results',
 'heracles_results',
 'heracles_results_dist',
 'ir_analysis_dist',
 'ir_analysis_result',
 'ir_analysis_strata_stats',
 'ir_strata',
 'heracles_periods',
 'cc_results',
 'pathway_analysis_codes',
 'pathway_analysis_events',
 'pathway_analysis_paths',
 'pathway_analysis_stats',
 'concept_hierarchy',
 'achilles_analysis',
 'achilles_results']

In [3]:
if CDM_SCHEMA in sa.inspect(engine).get_schema_names():
    print('WARNING: the schema already exists. Proceed with care')

## Create CDM

In [15]:
con = engine.connect()

In [6]:
# con.execute('DROP SCHEMA %s CASCADE;' % CDM_SCHEMA)
con.execute('CREATE SCHEMA %s;' % CDM_SCHEMA)

ProgrammingError: (psycopg2.errors.DuplicateSchema) schema "erspc" already exists

[SQL: CREATE SCHEMA erspc;]
(Background on this error at: http://sqlalche.me/e/f405)

In [16]:
con.execute('SET search_path TO %s;' % CDM_SCHEMA)
with open('pioneer_omop_cdm/OMOP CDM postgresql v6_0_onco_modified ddl.sql') as f_ddl:
    con.execute(f_ddl.read())

## Load data

In [9]:
from extract_inserts_from_sql_dump import extract_inserts

In [11]:
with open(CDM_DATA_PATH) as f_sql:
    queries = extract_inserts(f_sql)

Table attribute_definition      with       0 rows
Table care_site                 with       1 rows
Table cdm_source                with       0 rows
Table cohort                    with       0 rows
Table cohort_attribute          with       0 rows
Table cohort_definition         with       0 rows
Table condition_era             with       0 rows
Table condition_occurrence      with  22,960 rows
Table cost                      with       0 rows
Table death                     with  18,045 rows
Table device_exposure           with       0 rows
Table dose_era                  with       0 rows
Table drug_era                  with       0 rows
Table drug_exposure             with   2,941 rows
Table episode                   with       0 rows
Table episode_event             with       0 rows
Table fact_relationship         with       0 rows
Table location                  with       1 rows
Table location_history          with       0 rows
Table measurement               with 102,238 rows


In [17]:
con.execute('SET search_path TO %s;' % CDM_SCHEMA)
for query in queries:
    if 'visit_occurrence_source_value' in query:
        # Quick and dirty fix for visit occurrence source value. 
        # TODO: this field should not be exported (removed in final ETL step)
        import re
        query = query.replace(', visit_occurrence_source_value', '')
        if query.startswith('INSERT INTO observation'):
            query, n = re.subn(r'\(((?:.+?,){13}).+?,((?:.+?,){7})', r'(\1\2', query)
        else:
            query, n = re.subn(r", ?'[^']+?'\)", ')', query)
    try:
        r = con.execute(query)
        print(query[:20], ' ', r.rowcount)
    except Exception as e:
        print('Failed insert: %s - %s' % (query[:20], exc.args[0]))
print('Inserts done')

INSERT INTO care_sit   1
INSERT INTO conditio   22960
INSERT INTO death (p   18045
INSERT INTO drug_exp   2941
INSERT INTO location   1
INSERT INTO measurem   102238
INSERT INTO observat   167872
INSERT INTO observat   42375
INSERT INTO person (   42375
INSERT INTO procedur   14857
INSERT INTO specimen   10592
INSERT INTO visit_oc   56399
Inserts done


## Apply constraints, indexes and add vocab views

In [18]:
con.execute('SET search_path TO %s;' % CDM_SCHEMA)
with open('pioneer_omop_cdm/OMOP CDM postgresql v6_0_dev pk indexes.sql') as f_ind:
    con.execute(f_ind.read())

In [ ]:
with open('pioneer_omop_cdm/OMOP CDM postgresql v6_0_dev constraints.sql') as f_constr:
    con.execute(f_constr.read())

In [19]:
con.execute('SET search_path TO %s;' % CDM_SCHEMA)
with open('other/vocab_view.sql') as f_vocab_view:
    con.execute(f_vocab_view.read())

## Setup results schema

In [161]:
# con.execute('DROP SCHEMA %s CASCADE;' % RESULTS_SCHEMA)
con.execute('CREATE SCHEMA %s;' % RESULTS_SCHEMA)

In [170]:
con.execute('SET search_path TO %s;' % RESULTS_SCHEMA)
with open('other/results_ddl_hierarchy_2.7.4.sql') as f_results:
    con.execute(f_results.read())

Somehow Achilles doesn't create the results_dist table, copy from other results schema

In [27]:
con.execute('CREATE TABLE %s.achilles_results_dist AS (SELECT * FROM synpuf_1k_results.achilles_results_dist) WITH NO DATA' % CDM_SCHEMA)

In [28]:
con.close()

## Source Daimon

_manual step_

To add in Atlas->Configuration

Note: had to 'Clear Configuration Cash' to make it work. Gettign EntityExsistsErrors before that

## Run Achilles (R)

Run Achilles R script